In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

import matplotlib.pyplot as plt


In [2]:
device = torch.device("cuda")

In [3]:
BATCH_SIZE = 32

In [30]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.1307,), (0.3081,))])


trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE,
                                         shuffle=False, num_workers=2)

In [ ]:
trainset.data.shape

In [6]:
examples = enumerate(testloader)
idx, (data,label) = next(examples)

In [7]:
data.shape

torch.Size([32, 1, 28, 28])

In [ ]:
fig = plt.figure(figsize=(12, 12))
for i in range(9):
  plt.subplot(3,3, i + 1)
  plt.tight_layout()
  plt.imshow(data[i][0], cmap='gray', interpolation='none')
  plt.title(f"Ground truth: {label[i]}")
  plt.xticks([])
  plt.yticks([])

In [9]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        ## Warstwa konwolucyjna
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1, padding=0)
        ## Warstwa max pooling 
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.pool2 = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(256, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
net = Net().to(device)
net

In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [ ]:
net.train()
for epoch in range(5): 

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print('[%d/5] loss: %.3f' %
          (epoch+1 ,  running_loss / 2000))
    running_loss = 0.0

print('Finished Training')

In [13]:
import numpy as np
def imshow(img, img_number, labels):
  fig = plt.figure(figsize=(18, 18))
  for i in range(img_number):
    plt.subplot(int(img_number**0.5),int(img_number**0.5), i + 1)
    plt.tight_layout()
    plt.imshow(img[i][0], cmap='gray', interpolation='none')
    plt.title(f"Ground truth: {labels[i]}")
    plt.xticks([])
    plt.yticks([])

In [ ]:
numb_examples = 16
examples = enumerate(testloader)
idx, (images, labels) = next(examples)
images = images[:numb_examples]

# print images
imshow(images, numb_examples, labels)

In [ ]:
net.eval()
outputs = net(images.to(device))
outputs

In [ ]:
_, predicted = torch.max(outputs, 1)

print('Predicted:    ', ' '.join('%5s' % int(predicted[j])
                              for j in range(numb_examples)))
print('Ground truth: ', ' '.join('%5s' % int(labels[j])
                              for j in range(numb_examples)))

In [ ]:
corrects = 0
total = 0
net.eval()
for images, labels in testloader:
  images = images.to(device)
  labels = labels.to(device)
  outputs = net(images)
  _, predicted = torch.max(outputs, 1)
  corrects += (labels == predicted).sum().item()
  total += len(labels)
print(f"Accuracy: {corrects / total}")

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
from PIL import Image
from io import BytesIO
img = Image.open(BytesIO(uploaded['image.png'])).convert('L')
img = np.asarray(img)
plt.imshow(img)
plt.show()

In [ ]:
type(img)

In [257]:
transform = transforms.Compose(
  [transforms.ToTensor(),
     transforms.Normalize((0.1307,), (0.3081,))
])

In [258]:
img_tensor = transform(img.astype(np.float32))
img_tensor = torch.unsqueeze(img_tensor, dim=0)

In [ ]:
plt.imshow(img_tensor[0][0], cmap='gray', interpolation='none')
plt.show()

In [ ]:
img_tensor.shape

In [ ]:
net.eval()
outputs = net(img_tensor.to(device))
outputs

In [ ]:
_, predicted = torch.max(outputs, 1)
print(f"Predicted: {predicted[0]}")